# Loading

In [21]:
import tobii_research as tr
from utils import *

def get_tracker():
  all_eyetrackers = tr.find_all_eyetrackers()

  for tracker in all_eyetrackers:
    print("Model: " + tracker.model)
    print("Serial number: " + tracker.serial_number) 
    print(f"Can stream eye images: {tr.CAPABILITY_HAS_EYE_IMAGES in tracker.device_capabilities}")
    print(f"Can stream gaze data: {tr.CAPABILITY_HAS_GAZE_DATA in tracker.device_capabilities}")
    return tracker

tracker = get_tracker()

data, dict = build_dataset(tracker, 'title')

data

Model: Tobii Pro Fusion
Serial number: TPFC6-010203104285
Can stream eye images: True
Can stream gaze data: True
{'device_time_stamp': 1443910787, 'system_time_stamp': 333630782536, 'left_gaze_point_on_display_area': (0.03616883605718613, 0.030308803543448448), 'left_gaze_point_in_user_coordinate_system': (-87.5103530883789, 115.5667953491211, 31.421062469482422), 'left_gaze_point_validity': 1, 'left_pupil_diameter': 6.72967529296875, 'left_pupil_validity': 1, 'left_gaze_origin_in_user_coordinate_system': (-78.22499084472656, -33.577457427978516, 664.71826171875), 'left_gaze_origin_in_trackbox_coordinate_system': (0.6581600904464722, 0.571628749370575, 0.492565780878067), 'left_gaze_origin_validity': 1, 'right_gaze_point_on_display_area': (0.08352570235729218, 0.046611104160547256), 'right_gaze_point_in_user_coordinate_system': (-78.57560729980469, 113.76038360595703, 30.763582229614258), 'right_gaze_point_validity': 1, 'right_pupil_diameter': 6.556427001953125, 'right_pupil_validity':

,device_time_stamp,system_time_stamp,left_gaze_point_on_display_area,left_gaze_point_in_user_coordinate_system,left_gaze_point_validity,left_pupil_diameter,left_pupil_validity,left_gaze_origin_in_user_coordinate_system,left_gaze_origin_in_trackbox_coordinate_system,left_gaze_origin_validity,right_gaze_point_on_display_area,right_gaze_point_in_user_coordinate_system,right_gaze_point_validity,right_pupil_diameter,right_pupil_validity,right_gaze_origin_in_user_coordinate_system,right_gaze_origin_in_trackbox_coordinate_system,right_gaze_origin_validity,type
timestep_1,1443910787,333630782536,"(0.03616883605718613, 0.030308803543448448)","(-87.5103530883789, 115.5667953491211, 31.4210...",1,6.729675,1,"(-78.22499084472656, -33.577457427978516, 664....","(0.6581600904464722, 0.571628749370575, 0.4925...",1,"(0.08352570235729218, 0.046611104160547256)","(-78.57560729980469, 113.76038360595703, 30.76...",1,6.556427,1,"(-13.864234924316406, -41.66706848144531, 668....","(0.5278940200805664, 0.587233304977417, 0.4916...",1,title
timestep_2,1444419144,333631290890,"(0.03714146465063095, 0.03058266080915928)","(-87.32685089111328, 115.53643798828125, 31.41...",1,6.665344,1,"(-78.89230346679688, -33.421592712402344, 664....","(0.6594666838645935, 0.5713006258010864, 0.493...",1,"(0.089113749563694, 0.046123359352350235)","(-77.52131652832031, 113.81442260742188, 30.78...",1,6.483185,1,"(-14.52080249786377, -41.41688537597656, 668.6...","(0.5292003154754639, 0.5867037773132324, 0.492...",1,title
timestep_3,1444927501,333631799244,"(0.05580023303627968, 0.044415876269340515)","(-83.8065185546875, 114.00363159179688, 30.852...",1,6.666611,1,"(-79.56069946289062, -33.89613342285156, 664.5...","(0.6608945727348328, 0.5722947120666504, 0.492...",1,"(0.10681717097759247, 0.07349632680416107)","(-74.18123626708984, 110.78133392333984, 29.67...",1,6.42308,1,"(-15.175649642944336, -41.74304962158203, 667....","(0.5305601954460144, 0.5874479413032532, 0.490...",1,title
timestep_4,1445427525,333632299264,"(0.12542901933193207, 0.089913509786129)","(-70.66976165771484, 108.96220397949219, 29.01...",1,6.602448,1,"(-79.46034240722656, -34.268131256103516, 664....","(0.6607397198677063, 0.5730674862861633, 0.492...",1,"(0.16018593311309814, 0.11381831765174866)","(-64.11222839355469, 106.31338500976562, 28.05...",1,6.459702,1,"(-15.041261672973633, -42.18033981323242, 667....","(0.5302805304527283, 0.5883171558380127, 0.490...",1,title
timestep_5,1445944215,333632815951,"(0.29066652059555054, 0.20405377447605133)","(-39.49464797973633, 96.31471252441406, 24.413...",1,6.573151,1,"(-79.21031188964844, -34.61151123046875, 663.8...","(0.6603296995162964, 0.5737989544868469, 0.491...",1,"(0.31987497210502625, 0.25527524948120117)","(-33.98392868041992, 90.63904571533203, 22.348...",1,6.425125,1,"(-14.844171524047852, -42.52239990234375, 667....","(0.5299218893051147, 0.5890824198722839, 0.489...",1,title
timestep_6,1446444236,333633315968,"(0.2754512131214142, 0.20919771492481232)","(-42.36529541015625, 95.74472045898438, 24.206...",1,6.496552,1,"(-79.4903335571289, -34.88529968261719, 663.86...","(0.660892903804779, 0.5743563771247864, 0.4911...",1,"(0.33837130665779114, 0.2808184027671814)","(-30.49424934387207, 87.8086929321289, 21.3179...",1,6.364899,1,"(-15.1018705368042, -42.694644927978516, 667.3...","(0.5304285287857056, 0.5894098877906799, 0.489...",1,title
timestep_7,1446935921,333633807648,"(0.4544215202331543, 0.3694082498550415)","(-8.599225044250488, 77.99235534667969, 17.745...",1,6.488541,1,"(-79.00878143310547, -35.05248260498047, 662.8...","(0.6601250767707825, 0.5747687816619873, 0.488...",1,"(0.5101298093795776, 0.46472427248954773)","(1.911173939704895, 67.43072509765625, 13.9009...",1,6.29953,1,"(-14.708410263061523, -43.08475875854492, 667....","(0.5296440124511719, 0.5902155637741089, 0.489...",1,title
timestep_8,1447477617,333634349340,"(0.47912174463272095, 0.37304070591926575)","(-3.9390785694122314, 77.58985137939453, 17.59...",1,6.

In [7]:
# import time

# def gaze_data_callback(gaze_data):
#   global global_gaze_data
#   global_gaze_data = gaze_data
  
# def gaze_data(eyetracker, wait_time=5):
#   global global_gaze_data

#   print("Getting data...")
#   eyetracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, gaze_data_callback, as_dictionary=True)

#   time.sleep(wait_time)

#   eyetracker.unsubscribe_from(tr.EYETRACKER_GAZE_DATA, gaze_data_callback)

#   return global_gaze_data
  
# data = gaze_data(tracker)
# data

Getting data...


{'device_time_stamp': 890952768,
 'system_time_stamp': 333077828558,
 'left_gaze_point_on_display_area': (0.17883707582950592,
  0.003639146452769637),
 'left_gaze_point_in_user_coordinate_system': (-60.59334182739258,
  118.52195739746094,
  32.49665832519531),
 'left_gaze_point_validity': 1,
 'left_pupil_diameter': 5.0716400146484375,
 'left_pupil_validity': 1,
 'left_gaze_origin_in_user_coordinate_system': (-192.70730590820312,
  -92.11940002441406,
  701.1041259765625),
 'left_gaze_origin_in_trackbox_coordinate_system': (0.8749958276748657,
  0.6833948493003845,
  0.6267613768577576),
 'left_gaze_origin_validity': 1,
 'right_gaze_point_on_display_area': (-0.013065677136182785,
  0.06947140395641327),
 'right_gaze_point_in_user_coordinate_system': (-96.79935455322266,
  111.22730255126953,
  29.841655731201172),
 'right_gaze_point_validity': 1,
 'right_pupil_diameter': 5.419677734375,
 'right_pupil_validity': 1,
 'right_gaze_origin_in_user_coordinate_system': (-129.60992431640625,
 

# Visualizations

In [24]:
from matplotlib import pyplot as plt

# plot coordinates (you can use this for testing or for funsies)
# takes in a dataframe (call build_dataset first)
#          and a column_name (string)

def translate2ScreenX(xcoord):
    return 2*xcoord - 1

def translate2ScreenY(ycoord):
    return -2*ycoord + 1

def plot_coordinates(dataframe, column_name):
    # extract x and y coordinates from the specified column
    x_values = [point[0] for point in dataframe[column_name]]
    y_values = [point[1] for point in dataframe[column_name]]
    x_values = list(map(translate2ScreenX, x_values))
    y_values = list(map(translate2ScreenY, y_values))

    
    # plot the coordinates
    plt.figure(figsize=(8, 6))
    plt.scatter(x_values, y_values, marker='o', label=column_name)
    plt.title(f'Plot of {column_name} coordinates')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    
    plt.axhline(0, color='black', linestyle='--', linewidth=1)
    plt.axvline(0, color='black', linestyle='--', linewidth=1)
    
    plt.ylim(-1, 1)
    plt.xlim(-1, 1)
    plt.show()
    
plot_coordinates(data, 'left_gaze_point_on_display_area')


TypeError: list indices must be integers or slices, not float

# Make eye detection functions

In [37]:
# example
# function should accept a dictionary of gaze data (look at what the gaze_data function returns)
# when we call build_dataset, we also make a call to gaze_data, so we'll be able to 
# modify build_dataset to check for movement detection at every timestep.

# things to consider:
# this sample code only takes in one row/timestep
# if we want to use acceleration or speed of eye movements,
# we need to look at multiple rows at a time

def detect_movement_example(data_dict):
    # data_dict should only have 1 row
    assert(len(data_dict) == 1)  # remove this in practice, just keeping it here for initial testing
    
    # get the first (and only) row in data_dict
    eye_data = data_dict.iloc[0]
    
    # get eye coordinates
    left_x, left_y = data.left_gaze_point_on_display_area[0]
    right_x, right_y = data.right_gaze_point_on_display_area[0]
    
    # do some kind of detection
    print(left_x)
    return (left_x >= 0)
    
# test your function
data2 = pd.DataFrame(data.iloc[0]).transpose()  # call build_dataset and get a row
detect_movement_example(data2)                  # call detect_movement


0.4760041832923889


True

In [ ]:
# functions
# left
# right
# up 
# down
# eye roll

In [1]:
# try incorporating your function into a modified build_dataset, 
# so you can test your movement detection function while recording data

# make modifications here:
def modified_build_dataset(tracker, label, add_on = False, df_orig = pd.DataFrame(), 
                  time_step_sec = 0.5, tot_time_min = 0.1):
    
    global global_gaze_data
    
    intervals = math.ceil((tot_time_min * 60) / time_step_sec)
    dict_list = []
    
    for _ in range(intervals):
        data = gaze_data(tracker, time_step_sec)
        dict_list.append(data)
    
    tot_dict = combine_dicts_with_labels(dict_list)
    df = pd.DataFrame(tot_dict).T
    df['type'] = label
        
    if add_on:
        df_new = pd.concat([df_orig, df])
        df_new = df_new.reset_index(drop=True)
        return df_new
    
    else:
        return df, dict_list

NameError: name 'pd' is not defined